In [ ]:
!yes | wget "https://drive.google.com/file/d/1tGIO4-IPNtxJ6RQMmykvAfY_B0AaLY5A/view?usp=drive_link"

--2024-04-28 12:50:01--  https://drive.google.com/file/d/1tGIO4-IPNtxJ6RQMmykvAfY_B0AaLY5A/view?usp=drive_link
Resolving drive.google.com (drive.google.com)... 74.125.134.100, 74.125.134.102, 74.125.134.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.134.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=drive_link’

view?usp=drive_link     [ <=>                ]  83.81K  --.-KB/s    in 0.007s  

2024-04-28 12:50:02 (12.6 MB/s) - ‘view?usp=drive_link’ saved [85823]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Mounted at /content/drive


# LOADING DATA FROM CSV FILES

In [ ]:
def remove_na_en(data):
    return data[data['en'].notna()]
def filter_data(data):
    data = remove_na_en(data)
    data = data[data['ma'].notna()]
    data = data[['en','ma']]
    return data
def append_data(data_path):
    with open(data_path) as fil:
        return pd.read_csv(fil,sep=',',header=None,names=["en","ma",""],skip_blank_lines=True,index_col=None)

def data_loading(data_path):
    data =append_data(data_path)
    return filter_data(data)

In [ ]:

train = data_loading("/content/drive/MyDrive/aksharantar_sampled/mar/mar_train.csv")
print(len(train))
test = data_loading("/content/drive/MyDrive/aksharantar_sampled/mar/mar_test.csv")
print(test)
print(len(test))
test_en=list(test['en'])
test_ma=list(test['ma'])
print(test_en)
print(test_ma)
val = data_loading("/content/drive/MyDrive/aksharantar_sampled/mar/mar_valid.csv")
print(len(val))



51200
              en         ma
0        heetler      हिटलर
1         kshama      क्षमा
2         jinkta     जिंकता
3        kaushik      कौशिक
4      jaadoogar     जादूगर
...          ...        ...
4091     devicha     देवीचा
4092      beebee       बेबी
4093    sukshama    सुक्ष्म
4094  julymadhye  जुलैमध्ये
4095      robins    रॉबिन्स

[4096 rows x 2 columns]
4096
['heetler', 'kshama', 'jinkta', 'kaushik', 'jaadoogar', 'chaalavanyaachaa', 'samaajkaaryaat', 'thandine', 'maalmtaadharkaanmdhey', 'tukdoji', 'shivsainikasathi', 'subojitne', 'poshanasathi', 'hotee', 'ushmaa', 'tondawar', 'mele', 'annapolis', 'bombay', 'aavadtya', 'adhyaksheey', 'shetkaryaanchyaabaabat', 'barai', 'sangneamulay', 'sankate', 'mets', 'mulam', 'tinada', 'antarvastrapaasun', 'durgech', 'mulachaa', 'harananche', 'kashtache', 'talhta', 'eeseen', 'dharmshikshanaache', 'shheffield', 'loid', 'rugnanvar', 'anuvadasathee', 'babhadgav', 'karanehee', 'sampalyaane', 'ekalaa', 'chakreewadalacha', 'vibhagiya', 'doghaanni

#TOKENIZE DATASET

In [ ]:
def print_paras(marathi_tokens,english_tokens):
  print(marathi_tokens)
  print(len(marathi_tokens))
  print(english_tokens)
  print(len(english_tokens))

def tokenize_uniquely(data):
    marathi = train['ma'].values
    marathi_tokens = set()

    def add_english_tokens(x):
      for ch in x:
            english_tokens.add(ch)
      return english_tokens

    english = train['en'].values
    english_tokens = set()
    def add_marathi_tokens(y):
        for ch in y:
            marathi_tokens.add(ch)
        return marathi_tokens
    for x,y in zip(english,marathi):
        english_tokens = add_english_tokens(x)
        marathi_tokens = add_marathi_tokens(y)
    def ret_sorted_eng_tokens():
      return sorted(list(english_tokens))
    marathi_tokens = sorted(list(marathi_tokens))
    english_tokens = ret_sorted_eng_tokens()
    return english_tokens, marathi_tokens
english_tokens, marathi_tokens  = tokenize_uniquely(train)
print_paras(marathi_tokens,english_tokens)

['ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ऍ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'ळ', 'व', 'श', 'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्']
63
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
26


#TOKEN MAP

In [ ]:
#REWRITE AGAIN

def tokenize_map(english_tokens, marathi_tokens ):
    def get_m_dict():
        return dict([(ch,i+1) for i,ch in enumerate(marathi_tokens)])
    def get_e_dict():
        return dict([(ch,i+1) for i,ch in enumerate(english_tokens)])
    def reverse_m_tokens(marathi_tokens):
        return dict([(i+1,ch) for i,ch in enumerate(marathi_tokens)])
    english_token_map = get_e_dict()
    marathi_token_map = get_m_dict()
    reverse_marathi_token_map = reverse_m_tokens(marathi_tokens)
    #Adding blank space and BOS and EOS token
    marathi_token_map['.']=66
    english_token_map[" "] = 0
    reverse_marathi_token_map[64]='<unk>'
    marathi_token_map['<unk>']=64

    marathi_token_map[';']=65
    reverse_marathi_token_map[0]=''
    english_token_map[';']=27
    reverse_marathi_token_map[66]='.'
    marathi_token_map[" "] = 0
    reverse_marathi_token_map[65]=';'
    english_token_map['.']=28
    return english_token_map, marathi_token_map, reverse_marathi_token_map

eng_token_map, mar_token_map, reverse_marathi_token_map = tokenize_map(english_tokens, marathi_tokens)
print(mar_token_map)
print('mar:',len(mar_token_map))
print(reverse_marathi_token_map)
print('rev_mar:',len(reverse_marathi_token_map))
print(eng_token_map)
print('eng:',len(eng_token_map))

{'ँ': 1, 'ं': 2, 'ः': 3, 'अ': 4, 'आ': 5, 'इ': 6, 'ई': 7, 'उ': 8, 'ऊ': 9, 'ऋ': 10, 'ऍ': 11, 'ए': 12, 'ऐ': 13, 'ऑ': 14, 'ओ': 15, 'औ': 16, 'क': 17, 'ख': 18, 'ग': 19, 'घ': 20, 'च': 21, 'छ': 22, 'ज': 23, 'झ': 24, 'ञ': 25, 'ट': 26, 'ठ': 27, 'ड': 28, 'ढ': 29, 'ण': 30, 'त': 31, 'थ': 32, 'द': 33, 'ध': 34, 'न': 35, 'प': 36, 'फ': 37, 'ब': 38, 'भ': 39, 'म': 40, 'य': 41, 'र': 42, 'ल': 43, 'ळ': 44, 'व': 45, 'श': 46, 'ष': 47, 'स': 48, 'ह': 49, '़': 50, 'ा': 51, 'ि': 52, 'ी': 53, 'ु': 54, 'ू': 55, 'ृ': 56, 'ॅ': 57, 'े': 58, 'ै': 59, 'ॉ': 60, 'ो': 61, 'ौ': 62, '्': 63, '.': 66, '<unk>': 64, ';': 65, ' ': 0}
mar: 67
{1: 'ँ', 2: 'ं', 3: 'ः', 4: 'अ', 5: 'आ', 6: 'इ', 7: 'ई', 8: 'उ', 9: 'ऊ', 10: 'ऋ', 11: 'ऍ', 12: 'ए', 13: 'ऐ', 14: 'ऑ', 15: 'ओ', 16: 'औ', 17: 'क', 18: 'ख', 19: 'ग', 20: 'घ', 21: 'च', 22: 'छ', 23: 'ज', 24: 'झ', 25: 'ञ', 26: 'ट', 27: 'ठ', 28: 'ड', 29: 'ढ', 30: 'ण', 31: 'त', 32: 'थ', 33: 'द', 34: 'ध', 35: 'न', 36: 'प', 37: 'फ', 38: 'ब', 39: 'भ', 40: 'म', 41: 'य', 42: 'र', 43: 'ल', 44: 'ळ', 45: 'व

#MAXIMUM WORD LENGTH THAT ARE PRESENT IN THE DATASET




In [ ]:
def get_x_value(test):
  x=test['en'].values
  x=';'+ x + '.'
  return x
x = get_x_value(test)

def get_max_eng_len(x):
  return max([len(i) for i in x])

def get_y_value(test):
  y = test['ma'].values
  y = ';'+y+'.'
  return y
y=get_y_value(test)
#Getting max length
max_eng_len = get_max_eng_len(x)

def get_max_mar_len(y):
  return max([len(i) for i in y])

max_mar_len = get_max_mar_len(y)
print(max_eng_len,max_mar_len)

30 22


# ONE HOT ENCODING/EMBEDDING OUR INPUT



In [ ]:

import torch
#unknown token present in validation set as 'r.(in marathi)'
unknown_token=64
def print_train(train_process):
  print('\n')
  print('num of rows:',len(train_process))
  print('num of columns:',len(train_process[0]))
  print(train_process[0][1])

def data_cleaning(data):
    x,y = data['en'].values, data['ma'].values
    x = ";" + x + "."
    y = ";" + y + "."
    return (x,y)




def data_processing(data):
    x,y=data_cleaning(data)
    print(x[0:3])
    print(y[0:3])

    def get_a(xx):
      for j,ch in enumerate(xx):
          a[i,j] = eng_token_map[ch]
      return a



    a = torch.zeros((len(x),max_eng_len),dtype=torch.int64)
    print(a.shape)
    def get_b(yy):
      for j,ch in enumerate(yy):
            if ch not in mar_token_map:
              b[i,j]= unknown_token
            else:
              b[i,j] = mar_token_map[ch]
      return b

    b = torch.zeros((len(y),max_eng_len),dtype=torch.int64)

    data=[]
    def lamda_for_x(data,x):
      data = [(a[i], b[i]) for i in range(len(x))]
      return data

    for i,(xx,yy) in enumerate(zip(x,y)):
        a=get_a(xx)
        b=get_b(yy)
    print(a.shape)
    print(b.shape)
    data = lamda_for_x(data,x)
    return data

train_process=data_processing(train)
val_process=data_processing(val)
test_process=data_processing(test)
#print(train_process.shape)
print_train(train_process)



[';fusharun.' ';bhulthapana.' ';vhayaki.']
[';फुशारुन.' ';भूलथापाना.' ';व्हायकी.']
torch.Size([51200, 30])
torch.Size([51200, 30])
torch.Size([51200, 30])
[';garvyabarobarach.' ';reo.' ';sangrahalaye.']
[';गारव्याबरोबरच.' ';रियो.' ';संग्रहालये.']
torch.Size([4096, 30])
torch.Size([4096, 30])
torch.Size([4096, 30])
[';heetler.' ';kshama.' ';jinkta.']
[';हिटलर.' ';क्षमा.' ';जिंकता.']
torch.Size([4096, 30])
torch.Size([4096, 30])
torch.Size([4096, 30])


num of rows: 51200
num of columns: 2
tensor([65, 37, 54, 46, 51, 42, 54, 35, 66,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


In [ ]:
#Used later for reading the words
#This cell is to be checked---------------------------------------------
from torch import tensor
def lamda_1(data):
  return [int(i) for i in data]

def lamda_2(data):
  p_s = [reverse_marathi_token_map[idx] for idx in data]
  p_s = ''.join(p_s)
  return p_s
def reverse_tokenize(data):
  data=lamda_1(data)
  predicted_seq=lamda_2(data)
  return predicted_seq


#DATA LOADER

In [ ]:
#importing the necessary lilbraries
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch
BATCH_SIZE = 16
PAD_IDX = 0

def check_GPU():
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  return device
device=check_GPU()
BOS_IDX = ';'
EOS_IDX = '.'
shuffle=False
valid_iter = DataLoader(val_process, batch_size=BATCH_SIZE,shuffle=False)
train_iter = DataLoader(train_process, batch_size=BATCH_SIZE,shuffle=False)
print(len(train_iter))
test_iter = DataLoader(test_process, batch_size=BATCH_SIZE,shuffle=False)
print(len(test_iter))
print(len(valid_iter))

3200
256
256


In [ ]:
import string
import torch
import random
import torch.nn as nn
import os
from collections import Counter
import torch.optim as optim
# Set random seed for reproducibility
#giving a seed value for the random number
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)


In [ ]:
#installing wandb and login with my key
!pip install --upgrade wandb
import wandb
!wandb login fbf80504ccef17f5f3b05723be7ea4caff805164


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


#ATTENTION MODEL

In [ ]:
import torch.optim as optim
import random
from typing import Tuple
from torch import Tensor
import torch.nn.functional as F
class Encoder(nn.Module):
    def __init__(self, input_dim: int, emb_dim: int, enc_hid_dim: int, dec_hid_dim: int, dropout: float):
        def intializer(a):
          self.a=a
        super().__init__()
        def set_emb_dim():
           self.emb_dim = emb_dim
        set_emb_dim()
        self.enc_hid_dim = enc_hid_dim
        self.input_dim = input_dim
        def set_hid_dim():
          self.dec_hid_dim = dec_hid_dim
        set_hid_dim()
        self.embedding = nn.Embedding(self.input_dim, self.emb_dim)
        def set_dropout():
          self.dropout = dropout
        set_dropout()
        def set_rnn():
          self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        set_rnn()
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
    def ret_src(self,src):
      return src.permute(1,0)

    def forward(self, src: Tensor) -> Tuple[Tensor]:
        src = self.ret_src(src)
        def cal_tan(fc,hidden):
          return torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        hidden = cal_tan(self.fc,hidden)
        return outputs, hidden


class Attention(nn.Module):
    def __init__(self, enc_hid_dim: int, dec_hid_dim: int, attn_dim: int):
        def intializer(a):
            self.a=a
        super().__init__()
        def set_hid_dim():
          self.enc_hid_dim = enc_hid_dim
          self.dec_hid_dim = dec_hid_dim
        set_hid_dim()
        def set_attn_in():
          self.attn_in = (enc_hid_dim * 2) + dec_hid_dim
        set_attn_in()
        def attn():
          self.attn = nn.Linear(self.attn_in, attn_dim)
        attn()

    def ret_src_len(self,encoder_outputs):
        return encoder_outputs.shape[0]
    def forward(self, decoder_hidden: Tensor, encoder_outputs: Tensor) -> Tensor:
        src_len = self.ret_src_len(encoder_outputs)
        def ret_tanh():
          return torch.tanh(self.attn(torch.cat((repeated_decoder_hidden,encoder_outputs), dim = 2)))
        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)
        def ret_encoder_output():
            return encoder_outputs.permute(1, 0, 2)
        encoder_outputs = ret_encoder_output()
        energy = ret_tanh()
        def cal_softmax(attention):
          return F.softmax(attention, dim=1)
        attention = torch.sum(energy, dim=2)
        return cal_softmax(attention)


class Decoder(nn.Module):
    def __init__(self, output_dim: int, emb_dim: int, enc_hid_dim: int, dec_hid_dim: int, dropout: int, attention: nn.Module):
        def intializer(a):
            self.a=a
        super().__init__()
        def set_emb_dim():
          self.emb_dim = emb_dim
        set_emb_dim()
        self.enc_hid_dim = enc_hid_dim
        def set_output_dim():
          self.output_dim = output_dim
        set_output_dim()
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        self.attention = attention
        def set_rnn():
          return nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = set_rnn()
        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)
        def set_dropout():
          self.dropout = nn.Dropout(dropout)
        set_dropout()


    def _weighted_encoder_rep(self, decoder_hidden: Tensor, encoder_outputs: Tensor) -> Tensor:
        def set_attention():
          return self.attention(decoder_hidden, encoder_outputs)
        a = set_attention()
        a = a.unsqueeze(1)
        def ret_weighted_enc():
          return weighted_encoder_rep.permute(1, 0, 2)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted_encoder_rep = torch.bmm(a, encoder_outputs)
        weighted_encoder_rep = ret_weighted_enc()
        return weighted_encoder_rep


    def forward(self, input: Tensor, decoder_hidden: Tensor,encoder_outputs: Tensor) -> Tuple[Tensor]:
        def check_for(a,t,y):
          if(a>t):
             return a
          else:
             return y
        input = input.unsqueeze(0)
        def ret_embedded():
          return self.dropout(self.embedding(input))
        input = input.permute(1,0)
        embedded = ret_embedded()
        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden, encoder_outputs)

        #print(weighted_encoder_rep.shape)
        embedded = embedded.permute(1,0,2)
        def ret_rnn():
            return self.rnn(rnn_input, decoder_hidden.unsqueeze(0))
        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)
        output, decoder_hidden = ret_rnn()
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)
        def ret_for():
          return output, decoder_hidden.squeeze(0)
        output = self.out(torch.cat((output, weighted_encoder_rep, embedded), dim = 1))
        return ret_for()


class Seq2Seq(nn.Module):
    def __init__(self, encoder: nn.Module, decoder: nn.Module, device: torch.device):
        def intializer(a):
          self.a=a
        super().__init__()
        self.device = device
        self.encoder = encoder
        self.decoder = decoder

    def ret_b_size(self,src):
      return src.shape[0]


    def get_attention_weights(self, src):
        # Switch to evaluation mode
        self.eval()

        # Extract encoder outputs and hidden states
        with torch.no_grad():
            encoder_outputs, hidden = self.encoder(src)

            # Forward pass of the decoder
            attention_weights = []
            decoder_hidden = hidden
            for t in range(src.shape[1]):  # Assuming src is of shape [sequence_length, batch_size]
                output, decoder_hidden = self.decoder(src[:, t], decoder_hidden, encoder_outputs)
                attention_weights.append(decoder_hidden.squeeze(0))  # Assuming attention weights are from decoder hidden states

        # Return attention weights
        return attention_weights


    def forward(self,src: Tensor,trg: Tensor, teacher_forcing_ratio: float = 0.5) -> Tensor:
        batch_size =  self.ret_b_size(src)
        def ret_set_zeros():
          return torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        max_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        def ret_ecoder_src():
          return self.encoder(src)
        outputs = ret_set_zeros()
        #ATTENTION HEAT MAP METHOD
        #attentions = torch.zeros(max_len, batch_size, src.shape[1]).to(self.device)

        encoder_outputs, hidden = ret_ecoder_src()

        # first input to the decoder is the <sos> token
        trg = trg.permute(1,0)
        output = trg[0,:]
        def ret_teacher_force():
          return random.random() < teacher_forcing_ratio
        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            def ret_set_output():
              return (trg[t] if teacher_force else top1)
            teacher_force = ret_teacher_force()
            top1 = output.max(1)[1]
            output = ret_set_output()

        return outputs



INPUT_DIM = len(eng_token_map)
DEC_HID_DIM = 256
ENC_DROPOUT = 0.3
OUTPUT_DIM = len(mar_token_map)
DEC_EMB_DIM = 512
ENC_HID_DIM = 256
DEC_DROPOUT = 0.3
ATTN_DIM = 256
ENC_EMB_DIM = 512

def set_Encoder_config():
  return Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
enc = set_Encoder_config()
def set_Attention_config():
  return Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)
attn = set_Attention_config()
def set_Decoder_config():
  return Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)
dec = set_Decoder_config()
model = Seq2Seq(enc, dec, device).to(device)

def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' not in name:
            nn.init.constant_(param.data, 0)
        else:
          nn.init.normal_(param.data, mean=0, std=0.01)


#added custom weights
def get_optimizer():
  return optim.Adam(model.parameters())
model.apply(init_weights)

optimizer = get_optimizer()
def printPCount(model):
  print(f'The model has {total_parameters_count(model):,} trainable parameters')

def total_parameters_count(model: nn.Module):
    def utility1(a,b):
      if(a==1):
        return b.shape()
      else:
        return 0;
    return sum(p.numel() for p in model.parameters() if p.requires_grad)





printPCount(model)

The model has 2,630,467 trainable parameters


In [ ]:

def ret_src(src):
    max_len = max(len(word) for word in src)
    src_tensor = torch.tensor([[eng_token_map[char] if len(char) > 0 else 0 for char in word.ljust(max_len)] for word in src])  # Convert src list to tensor
    return src_tensor.permute(1, 0)  # Permute dimensions
def plot_attention(attention, input_word, file_name):
    fig = plt.figure(figsize=(3, 3))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    ax.set_xticklabels([''] + list(input_word), rotation=90)
    ax.set_yticklabels([''] + list(input_word))
    plt.savefig(file_name)
    plt.show()

# Assuming model is your trained Seq2Seq model
def get_attention_weights(model, src):
    src_tensor = ret_src(src)
    # Assuming model.get_attention_weights() extracts attention weights for src_tensor
    attention_weights = model.get_attention_weights(src_tensor)
    return attention_weights

# Assuming visualize_attention_heatmaps() visualizes attention heatmaps
def visualize_attention_heatmaps(attention_weights, src_tokens, num_inputs=10):
    random_indices = range(min(num_inputs, len(src_tokens)))
    for i in random_indices:
        attention = attention_weights[i].detach().numpy()
        input_word = src_tokens[i]
        plot_attention(attention, input_word, f"heatmap_example_{i}.png")

# Get attention weights
attention_weights = get_attention_weights(model, test_en)
# Visualize attention heatmaps
visualize_attention_heatmaps(attention_weights, test_en)

# ATTENTION WANDB IMPLEMENTATION

In [ ]:
import time
import math
sweep_config = {
    'sweep_id':1,
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'attn_dim':{
            'values':[64,128,256]
        },

        'dropout':{
            'values':[0.3,0.5,0.6]
        },
        'emb_dim': {
            'values': [128,256,512]
        },
        'hidden_layer_dim': {
            'values': [128,256,512]
        },

    }
}

sweep_id = wandb.sweep(sweep_config, entity='cs23m030', project="Assignment_3_DL_Testing")

def sweep_train():
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'dropout':0.6,
      'emb_dim':256,
      'attn_dim':128,
      'hidden_layer_dim':256,

  }

  # Initialize a new wandb run
  def ret_hid_dim():
    return wandb.config.hidden_layer_dim
  wandb.init(project='Assignment_3_DL_Testing', entity='cs23m030',config=config_defaults)
  def emb_dim():
    return wandb.config.emb_dim


  wandb.run.name = 'emb_dim:'+ str(emb_dim())+' ;hl:'+str(ret_hid_dim())+ ' ;attn_dim:'+str(wandb.config.attn_dim)+ ' ;dropout:'+str(wandb.config.dropout)
  config = wandb.config
  def emb_dim_write():
    return config.emb_dim
  emb_dim = emb_dim_write()
  def ret_config_hid():
      return config.hidden_layer_dim
  hidden_layer_dim = ret_config_hid()
  attn_dim = config.attn_dim
  dropout = config.dropout
  # Model training here
  def set_ID():
      return len(eng_token_map)
  INPUT_DIM = set_ID()
  OUTPUT_DIM = len(mar_token_map)
  def getAttention():
    return Attention(hidden_layer_dim, hidden_layer_dim, attn_dim)
  enc = Encoder(INPUT_DIM, emb_dim, hidden_layer_dim, hidden_layer_dim, dropout)

  attn = getAttention()

  dec = Decoder(OUTPUT_DIM, emb_dim, hidden_layer_dim, hidden_layer_dim, dropout, attn)

  model = Seq2Seq(enc, dec, device).to(device)
  model.apply(init_weights)
  def set_optim():
    return optim.Adam(model.parameters())
  optimizer = set_optim()
  criterion = nn.CrossEntropyLoss()

  model.train()

  def set_zero():
      return 0
  val_epoch_acc = set_zero()
  train_epoch_acc = set_zero()
  val_epoch_loss = set_zero()
  train_epoch_loss = set_zero()


  N_EPOCHS = 10
  CLIP = 1

  epoch=0
  while epoch < (N_EPOCHS):
  #TRAINING BLOCK
    model.train()
    for _, (src, trg) in enumerate(train_iter):
        def get_output_model(src,trg):
          return model(src, trg)
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = get_output_model(src,trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg.permute(1,0)
        trg = torch.reshape(trg[1:], (-1,))
        loss = criterion(output, trg)
        # calculate accuracy
        preds = torch.argmax(output, dim=1)
        non_pad_elements = (trg != 0).nonzero(as_tuple=True)[0]
        train_correct = preds[non_pad_elements] == trg[non_pad_elements]
        train_epoch_acc += train_correct.sum().item() / len(non_pad_elements)
        def loss_item_cur(loss):
          return loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)

        optimizer.step()

        train_epoch_loss += loss_item_cur(loss)

    def get_epoch_accuracy(train_epoch_acc):
        return train_epoch_acc / len(train_iter)
    train_epoch_loss = train_epoch_loss / len(train_iter)
    train_epoch_acc = get_epoch_accuracy(train_epoch_acc)


    #EVALUATION MODE
    model.eval()

    with torch.no_grad():

        for _, (src, trg) in enumerate(valid_iter):
            def get_model(src,trg):
              return model(src, trg, 0)
            src, trg = src.to(device), trg.to(device)

            output =  get_model(src,trg)#turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg.permute(1,0)
            trg = torch.reshape(trg[1:], (-1,))

            loss = criterion(output, trg)

            val_epoch_loss += loss.item()
            # calculate accuracy
            preds = torch.argmax(output, dim=1)
            non_pad_elements = (trg != 0).nonzero(as_tuple=True)[0]
            def epoch_val_acc(val_correct,non_pad_elements):
              return val_correct.sum().item() / len(non_pad_elements)

            val_correct = preds[non_pad_elements] == trg[non_pad_elements]
            val_epoch_acc += epoch_val_acc(val_correct,non_pad_elements)
    def get_val_epoch_acc(val_epoch_acc,valid_iter):
      return val_epoch_acc / len(valid_iter)

    def print_after_epoch(epoch, train_epoch_loss,train_epoch_acc ,val_epoch_loss,val_epoch_acc):
       print(f'Epoch: {epoch+1:02} |Train Loss: {train_epoch_loss:.3f} | Train accuracy: {train_epoch_acc:.3f}|Val. Loss: {val_epoch_loss:.3f} | Val accuracy: {val_epoch_acc:.3f}')
    val_epoch_loss = val_epoch_loss / len(valid_iter)
    val_epoch_acc = get_val_epoch_acc(val_epoch_acc,valid_iter)

    print_after_epoch(epoch, train_epoch_loss,train_epoch_acc ,val_epoch_loss,val_epoch_acc)
    wandb.log({"train_loss":train_epoch_loss,"train_accuracy": train_epoch_acc,"val_loss":val_epoch_loss,"val_accuracy":val_epoch_acc},)
    #emptying the cache after one complete run
    if epoch==N_EPOCHS-1:
            torch.cuda.empty_cache()
    epoch+=1

#RUNNING THE SWEEP
wandb.agent(sweep_id, function=sweep_train, count=1)

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Additional properties are not allowed ('sweep_id' was unexpected)


Create sweep with ID: k7upvhhi
Sweep URL: https://wandb.ai/cs23m030/Assignment_3_DL_Testing/sweeps/k7upvhhi


wandb: Agent Starting Run: cug306is with config:
wandb: 	attn_dim: 64
wandb: 	dropout: 0.5
wandb: 	emb_dim: 256
wandb: 	hidden_layer_dim: 128
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch: 01 |Train Loss: 1.106 | Train accuracy: 0.224|Val. Loss: 0.881 | Val accuracy: 0.214
Epoch: 02 |Train Loss: 0.763 | Train accuracy: 0.422|Val. Loss: 0.662 | Val accuracy: 0.392
Epoch: 03 |Train Loss: 0.509 | Train accuracy: 0.593|Val. Loss: 0.535 | Val accuracy: 0.524
Epoch: 04 |Train Loss: 0.383 | Train accuracy: 0.690|Val. Loss: 0.447 | Val accuracy: 0.626
Epoch: 05 |Train Loss: 0.264 | Train accuracy: 0.786|Val. Loss: 0.428 | Val accuracy: 0.672
Epoch: 06 |Train Loss: 0.222 | Train accuracy: 0.820|Val. Loss: 0.404 | Val accuracy: 0.699
Epoch: 07 |Train Loss: 0.199 | Train accuracy: 0.840|Val. Loss: 0.407 | Val accuracy: 0.706
Epoch: 08 |Train Loss: 0.183 | Train accuracy: 0.854|Val. Loss: 0.412 | Val accuracy: 0.709
Epoch: 09 |Train Loss: 0.172 | Train accuracy: 0.863|Val. Loss: 0.400 | Val accuracy: 0.723
Epoch: 10 |Train Loss: 0.162 | Train accuracy: 0.870|Val. Loss: 0.395 | Val accuracy: 0.726


train_accuracy,▁▃▅▆▇▇████
train_loss,█▅▄▃▂▁▁▁▁▁
val_accuracy,▁▃▅▇▇█████
val_loss,█▅▃▂▁▁▁▁▁▁
train_accuracy,0.87028
train_loss,0.16216
val_accuracy,0.72648
val_loss,0.39482


# VANILLA SEQ2SEQ MODEL

In [ ]:
criterion = nn.CrossEntropyLoss()

import time
import numpy as np
import math

def train(model: nn.Module, iterator: torch.utils.data.DataLoader, optimizer: optim.Optimizer, criterion: nn.Module, clip: float):
    model.train()
    def setZero():
      return 0

    correct_words = setZero()
    total_chars = setZero()
    epoch_acc = setZero()
    epoch_word_acc = setZero()
    total_words = setZero()
    epoch_loss = setZero()
    correct_chars = setZero()

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        def getOutput(output):
          return output[1:].view(-1, output.shape[-1])
        output = model(src, trg)
        output = getOutput(output)
        trg = trg.permute(1,0)
        trg = torch.reshape(trg[1:], (-1,))
        loss = criterion(output, trg)

        # calculate character level accuracy
        def getNonZero(trg):
          return (trg != 0).nonzero(as_tuple=True)[0]
        preds = torch.argmax(output, dim=1)
        non_pad_elements = getNonZero(trg)
        correct = preds[non_pad_elements] == trg[non_pad_elements]
        epoch_acc += correct.sum().item() / len(non_pad_elements)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        def calAccLoss(epoch_loss,epoch_acc):
          return epoch_loss / len(iterator), epoch_acc / len(iterator)

        optimizer.step()

        epoch_loss += loss.item()

    return calAccLoss(epoch_loss,epoch_acc)


def evaluate(model: nn.Module, iterator: torch.utils.data.DataLoader,criterion: nn.Module):
    def setZero():
        return 0
    model.eval()
    correct_words = setZero()
    total_chars = setZero()
    epoch_loss = setZero()
    epoch_acc = setZero()
    correct_chars = setZero()
    total_words = setZero()
    epoch_word_acc = setZero()
    solution=[]
    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            def get_src_trg():
              return src.to(device), trg.to(device)
            src, trg = get_src_trg()

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg.permute(1,0)
            trg = torch.reshape(trg[1:], (-1,))

            loss = criterion(output, trg)

            epoch_loss += loss.item()

            # calculate accuracy
            preds = torch.argmax(output, dim=1)
            non_pad_elements = (trg != 0).nonzero(as_tuple=True)[0]
            def cal_epoch_acc(correct,non_pad_elements):
              return  correct.sum().item() / len(non_pad_elements)

            correct = preds[non_pad_elements] == trg[non_pad_elements]
            epoch_acc +=cal_epoch_acc(correct,non_pad_elements)

            b=np.zeros((16,29))
            def cal_pred(b,preds):
              for j in range (29):
                 b[i][j]=preds[16*j+i]
            def utility_1(u,v):
              return u>v
            for i in range(16):
              cal_pred(b,preds)
            def calel(epoch_loss,epoch_acc):
              return epoch_loss / len(iterator), epoch_acc / len(iterator)
            for i in range(16):
              solution.append(reverse_tokenize(b[i]))
            loss_e,acc_e = calel(epoch_loss,epoch_acc)
    return loss_e, acc_e, solution

def ret_elapsed_secs(elapsed_time,elapsed_mins):
  return int(elapsed_time - (elapsed_mins * 60))
def epoch_time(start_time: int, end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = ret_elapsed_secs(elapsed_time,elapsed_mins)
    return elapsed_mins, elapsed_secs


N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')
epoch=0
while epoch < (N_EPOCHS):
    start_time = time.time()
    train_loss,train_acc = train(model, train_iter, optimizer, criterion, CLIP)
    def print_epoch_result(epoch,epoch_mins,epoch_secs,train_loss,train_acc,valid_loss,val_acc,solution_valid):
      print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
      print(f'\tTrain Loss: {train_loss:.3f} | Train accuracy: {train_acc:.3f}')
      print(f'\t Val. Loss: {valid_loss:.3f} | Val accuracy: {val_acc:.3f}')
      print(solution_valid[0])

    def get_ep_time(start_time,end_time):
      return epoch_time(start_time, end_time)
    valid_loss,val_acc,solution_valid = evaluate(model, valid_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = get_ep_time(start_time,end_time)
    print_epoch_result(epoch,epoch_mins,epoch_secs,train_loss,train_acc,valid_loss,val_acc,solution_valid)
    epoch+=1


def print_test_loss(test_loss,test_accuracy):
  print(f'| Test Loss: {test_loss:.3f} | Test accuracy: {test_accuracy:.3f} |')
test_loss,test_accuracy,solution_test_att = evaluate(model, test_iter, criterion)
print_test_loss(test_loss,test_accuracy)

Epoch: 01 | Time: 3m 46s
	Train Loss: 0.481 | Train accuracy: 0.621
	 Val. Loss: 0.493 | Val accuracy: 0.578
गरव्याबरोबरच.
Epoch: 02 | Time: 3m 46s
	Train Loss: 0.320 | Train accuracy: 0.743
	 Val. Loss: 0.458 | Val accuracy: 0.637
गरव्याबरोबरच.
Epoch: 03 | Time: 3m 44s
	Train Loss: 0.264 | Train accuracy: 0.787
	 Val. Loss: 0.446 | Val accuracy: 0.657
गरव्याबरोबरच.
Epoch: 04 | Time: 3m 43s
	Train Loss: 0.231 | Train accuracy: 0.814
	 Val. Loss: 0.428 | Val accuracy: 0.674
गरव्याबरोबरच.
Epoch: 05 | Time: 3m 42s
	Train Loss: 0.205 | Train accuracy: 0.835
	 Val. Loss: 0.424 | Val accuracy: 0.686
गारव्याबरोबरच.
Epoch: 06 | Time: 3m 44s
	Train Loss: 0.190 | Train accuracy: 0.849
	 Val. Loss: 0.446 | Val accuracy: 0.688
गरव्याबरोबरच.
Epoch: 07 | Time: 3m 46s
	Train Loss: 0.177 | Train accuracy: 0.859
	 Val. Loss: 0.441 | Val accuracy: 0.692
गरव्याबरोबरच.
Epoch: 08 | Time: 3m 44s
	Train Loss: 0.167 | Train accuracy: 0.868
	 Val. Loss: 0.454 | Val accuracy: 0.698
गरव्याबरोबरच.
Epoch: 09 | Tim

#VANILLA SEQ2SEQ MODEL ON TEST DATASET




In [ ]:
# Define the vocabulary of English and Devanagari characters


# Define the start and end of sequence tokens
BOS_token = 28
EOS_token = 29

def get_max_len():
  return 30
# Define the maximum sequence lengths for input and output sequences
MAX_LEN_EN = get_max_len()
MAX_LEN_MA = get_max_len()
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers=1 , cell_type="lstm", p=0.5):
        def intializer(a):
          self.a=a
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        def set_dropout():
          return nn.Dropout(p)
        self.cell_type=cell_type
        self.dropout=set_dropout()
        def set_embeddin():
          return nn.Embedding(input_size,embedding_size)
        self.embedding = set_embeddin()
        def get_lstm():
            return nn.LSTM(input_size, hidden_size, num_layers,dropout=p)
        def get_rnn():
            return nn.RNN(input_size, hidden_size, num_layers,dropout=p)
        def get_gru():
            return nn.GRU(input_size, hidden_size, num_layers,dropout=p)
        if cell_type == "lstm":
            self.rnn = get_lstm()
        elif cell_type == "rnn":
            self.rnn = get_rnn()
        elif cell_type == "gru":
            self.rnn = get_gru()
        else:
            raise ValueError("Invalid cell type selected: {}".format(cell_type))

    def forward(self, x):
        def ret_rnn(embedding):
            return self.rnn(embedding)

        def d_emb():
          return self.dropout(self.embedding(x))
        x=x.permute(1,0)
        embedding=d_emb()
        if self.cell_type=="lstm":
         outputs,(hidden,cell)=ret_rnn(embedding)
        else:
           hidden, cell = ret_rnn(embedding)
        return hidden, cell


class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1, cell_type="lstm",p=0.5):
        def intializer(a):
          self.a=a
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        def set_drop():
          return nn.Dropout(p)
        self.num_layers = num_layers
        self.dropout=set_drop()
        self.embedding=nn.Embedding(input_size, embedding_size)
        self.cell_type=cell_type
        self.output_size = output_size
        def set_gru():
          return nn.GRU(output_size, hidden_size, num_layers)
        def set_lstm():
          return nn.LSTM(embedding_size, hidden_size, num_layers,dropout=p)
        def set_rnn():
          return nn.RNN(output_size, hidden_size, num_layers)
        if cell_type == "gru":
            self.rnn = set_gru()
        elif cell_type == "lstm":
            self.rnn = set_lstm()
        elif cell_type == "rnn":
            self.rnn = set_rnn()
        else:
            raise ValueError("Invalid cell type selected: {}".format(cell_type))
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden,cell):
       # shape of x: (N) but we want (1,N)
       def lstm_config(embedding):
        return self.rnn(embedding, (hidden,cell))
       x=x.unsqueeze(0)

       embedding= self.dropout(self.embedding(x))
       #embedding shape: (1,N,hidden_size)
       def rnn_config(embedding):
          return self.rnn(embedding, hidden)
       if self.cell_type=="lstm":
        outputs,(hidden,cell) = lstm_config(embedding)
       else:
         outputs, hidden = rnn_config(embedding)
       #shape of outputs: (1,N,hidden_size)


       predictions=self.fc(outputs)
       #shape of predictions: (1,N,length_of_vocab)

       predictions = predictions.squeeze(0)
       def ret_pred_lstm():
         return predictions, hidden, cell
       def ret_pred_rnn():
         return predictions, hidden
       if self.cell_type=="lstm":
         return ret_pred_lstm()
       else:
         return ret_pred_rnn()

class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder,cell_type="lstm"):
        def intializer(a):
          self.a=a
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        def set_celltype():
          return cell_type
        self.cell_type = set_celltype()
    def get_vocab_size(self,decoder):
        return self.decoder.output_size
    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[0]
        def ret_outputs(batch_size,target_vocab_size,target_len):
          return torch.zeros(target_len,batch_size,target_vocab_size).to(device)

        target_len = target.shape[1]
        target_vocab_size=self.get_vocab_size(self.decoder)

        outputs=ret_outputs(batch_size,target_vocab_size,target_len)
        cell,hidden=self.encoder(source)
        def lstm_decoder():
            return self.decoder(x,hidden,cell)
        target = target.permute(1,0)
        x = target[0,:]
        def non_lstm_decoder():
            return self.decoder(x,hidden,cell)
        for t in range(1,target_len):
          if self.cell_type=="lstm":
            output,hidden,cell = lstm_decoder()
          else:
            output, hidden = non_lstm_decoder()

          outputs[t] = output
          def get_x():
            return target[t] if random.random() < teacher_force_ratio else best_guess
          best_guess = output.argmax(1)
          x= get_x()
        return outputs


In [ ]:
import numpy as np
def cpu_check():
  return torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs=10
learning_rate=0.001

device=cpu_check()
def set_input_size():
  return len(mar_token_map)
output_size=set_input_size()
input_size_encoder = len(eng_token_map)
input_size_decoder=set_input_size()
decoder_embedding_size=67
encoder_embedding_size=29
def set_d_out():
  return 0.2
cell_type="gru"
hidden_size=256
dec_dropout=set_d_out()
num_layers=4
def get_encoder_configs():
  return Encoder(input_size_encoder,encoder_embedding_size,hidden_size,num_layers,cell_type,p=enc_dropout).to(device)
enc_dropout=set_d_out()
#input_size, embedding_size, hidden_size, num_layers=1 , cell_type="lstm", p=0.5
encoder_net=get_encoder_configs()
def ret_loss_type():
  return nn.CrossEntropyLoss()
def get_decoder_configs():
  return Decoder(input_size_decoder,decoder_embedding_size,hidden_size,output_size,num_layers,cell_type,p=dec_dropout).to(device)
decoder_net=get_decoder_configs()
model=Seq2Seq(encoder_net,decoder_net,cell_type).to(device)
criterion = ret_loss_type()
import math
import time

optimizer = optim.Adam(model.parameters(),lr=learning_rate)
def get_initial_loss_acc():
  return 0
def train(model: nn.Module,iterator: torch.utils.data.DataLoader, optimizer: optim.Optimizer, criterion: nn.Module, clip: float):
    model.train()
    epoch_loss = get_initial_loss_acc()
    epoch_acc = get_initial_loss_acc()

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        def get_output():
            return model(src, trg)
        optimizer.zero_grad()

        output = get_output()
        output = output[1:].view(-1, output.shape[-1])
        def reshape_trg():
          return torch.reshape(trg[1:], (-1,))
        trg = trg.permute(1,0)
        trg = reshape_trg()
        loss = criterion(output, trg)
        def get_correctness(preds,trg):
          return preds[non_pad_elements] == trg[non_pad_elements]
        def check_fornonzero():
          return (trg != 0).nonzero(as_tuple=True)[0]
        # calculate accuracy
        preds = torch.argmax(output, dim=1)
        non_pad_elements = check_fornonzero()
        def calculate_accuracy(correct,non_pad_elements):
          return correct.sum().item() / len(non_pad_elements)
        correct = get_correctness(preds,trg)
        epoch_acc += calculate_accuracy(correct,non_pad_elements)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        def ret_E_loss(epoch_loss):
          return epoch_loss / len(iterator)
        optimizer.step()
        def ret_E_acc(epoch_acc):
          return epoch_acc / len(iterator)
        epoch_loss += loss.item()

    return ret_E_loss(epoch_loss),ret_E_acc(epoch_acc)



def evaluate(model: nn.Module, iterator: torch.utils.data.DataLoader, criterion: nn.Module):
    model.eval()
    def set_epoch_initials():
      return 0
    epoch_acc = set_epoch_initials()
    solution=[]
    epoch_loss = set_epoch_initials()

    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)
            def ret_output(output):
              return output[1:].view(-1, output.shape[-1])

            output = model(src, trg, 0) #turn off teacher forcing
            output = ret_output(output)
            def ret_loss_fun(output,trg):
              return criterion(output, trg)
            trg = trg.permute(1,0)
            trg = torch.reshape(trg[1:], (-1,))

            def cal_b_pred(preds,b):
              for j in range (29):
                 b[i][j]=preds[16*j+i]
              return preds,b
            loss = ret_loss_fun(output, trg)

            epoch_loss += loss.item()
            # calculate accuracy
            def ret_epoch_acc(correct,non_pad_elements):
              return correct.sum().item() / len(non_pad_elements)
            preds = torch.argmax(output, dim=1)
            b=np.zeros((16,29))
            def get_non_pad_elements(trg):
              return (trg != 0).nonzero(as_tuple=True)[0]
            for i in range(16):
              preds,b=cal_b_pred(preds,b)

            for i in range(16):
              solution.append(reverse_tokenize(b[i]))
            non_pad_elements = get_non_pad_elements(trg)

            correct = preds[non_pad_elements] == trg[non_pad_elements]
            epoch_acc += ret_epoch_acc(correct,non_pad_elements)

    e_loss=epoch_loss / len(iterator)
    e_acc=epoch_acc / len(iterator)
    return e_loss, e_acc ,solution


def epoch_time(start_time: int,end_time: int):
    def elapsed_sec(elapsed_time,elapsed_mins):
      return int(elapsed_time - (elapsed_mins * 60))
    elapsed_time = end_time - start_time
    def get_mins(elapsed_time):
      return int(elapsed_time / 60)
    elapsed_mins = get_mins(elapsed_time)
    elapsed_secs = elapsed_sec(elapsed_time,elapsed_mins)
    return elapsed_mins, elapsed_secs


N_EPOCHS = 10
best_valid_loss = float('inf')
def print_test_result(test_loss,test_accuracy):
  print(f'| Test Loss: {test_loss:.3f} | Test accuracy: {test_accuracy:7.3f} |')
CLIP = 1

epoch=0
while epoch < (N_EPOCHS):
    def get_train_paras():
      return train(model, train_iter, optimizer, criterion, CLIP)
    start_time = time.time()
    def do_evaluate():
      return evaluate(model, valid_iter, criterion)
    train_loss,train_acc = get_train_paras()
    def print_epoch_result(epoch,epoch_mins,epoch_secs,train_loss,train_acc,valid_loss,val_acc,solution):
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train accuracy: {train_acc:.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} | Val accuracy: {val_acc:.3f}')
        print(solution[0])
    valid_loss,val_acc,solution = do_evaluate()
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print_epoch_result(epoch,epoch_mins,epoch_secs,train_loss,train_acc,valid_loss,val_acc,solution)

    epoch+=1

test_loss,test_accuracy,solution_test = evaluate(model, test_iter, criterion)
print_test_result(test_loss,test_accuracy)


Epoch: 01 | Time: 2m 54s
	Train Loss: 0.821 | Train accuracy: 0.410
	 Val. Loss: 0.518 | Val accuracy: 0.569
गर्व्याबोबरहार.
Epoch: 02 | Time: 2m 52s
	Train Loss: 0.360 | Train accuracy: 0.718
	 Val. Loss: 0.464 | Val accuracy: 0.635
गरव्याबरोबरच.
Epoch: 03 | Time: 2m 53s
	Train Loss: 0.279 | Train accuracy: 0.779
	 Val. Loss: 0.446 | Val accuracy: 0.664
गरव्याबरोबरच.
Epoch: 04 | Time: 2m 54s
	Train Loss: 0.246 | Train accuracy: 0.805
	 Val. Loss: 0.426 | Val accuracy: 0.674
गरव्याबरोबरच.
Epoch: 05 | Time: 2m 54s
	Train Loss: 0.225 | Train accuracy: 0.822
	 Val. Loss: 0.415 | Val accuracy: 0.688
गरव्याबरोबरच.
Epoch: 06 | Time: 2m 55s
	Train Loss: 0.214 | Train accuracy: 0.830
	 Val. Loss: 0.430 | Val accuracy: 0.691
गरव्याबरोबरच.
Epoch: 07 | Time: 2m 55s
	Train Loss: 0.204 | Train accuracy: 0.838
	 Val. Loss: 0.401 | Val accuracy: 0.700
गरव्याबरोबरच.
Epoch: 08 | Time: 2m 53s
	Train Loss: 0.198 | Train accuracy: 0.843
	 Val. Loss: 0.426 | Val accuracy: 0.697
गरव्याबरोबरच.
Epoch: 09 | Ti

In [ ]:

!pip install --upgrade wandb
import wandb
!wandb login fbf80504ccef17f5f3b05723be7ea4caff805164 #my API key for wandb login


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# WANDB IMPLEMENTATION

In [ ]:
sweep_config = {
    'sweep_id':2,
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'hidden_layers':{
            'values':[64,128,256]
        },
        'cell_type':{
            'values':['rnn','lstm','gru']
        },

        'layer_size': {
            'values': [1,2,3,4]
        },

        'dropout':{
            'values':[0,0.2,0.4,0.6]
        },
         'lr': {
            'values': [0.001,0.0001]
        },
    }
}



sweep_id = wandb.sweep(sweep_config, entity='cs23m030', project="Assignment_3_DL_Testing")
def ret_dropout():
  return wandb.config.dropout
def ret_layer_size():
  return wandb.config.layer_size
def ret_lr():
  return wandb.config.lr
def sweep_train():
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'cell_type':'lstm',
      'lr':0.0001,
      'hidden_layers':128,
      'dropout':0.4,
      'layer_size':4,

  }
  def ret_cell_type():
    return wandb.config.cell_type
  # Initialize a new wandb run
  wandb.init(project='Assignment_3_DL_Testing', entity='cs23m030',config=config_defaults)
  wandb.run.name = 'cell:'+ str(ret_cell_type())+' ;lr:'+str(ret_lr())+ ' ;layer_size:'+str(ret_layer_size())+ ' ;dropout:'+str(ret_dropout())+' ;hidden:'+str(wandb.config.hidden_layers)
  def get_hidden_l():
    return config.hidden_layers
  config = wandb.config
  cell_type = config.cell_type
  hidden_layers = get_hidden_l()
  def get_dec():
    return Decoder(input_size_decoder,decoder_embedding_size,hidden_layers,output_size,layer_size,cell_type,p=dropout).to(device)
  lr = config.lr
  layer_size = config.layer_size
  dropout = config.dropout
  # Model training here
  def get_enc():
    return Encoder(input_size_encoder,encoder_embedding_size,hidden_layers,layer_size,cell_type,p=dropout).to(device)
  input_size_encoder = len(eng_token_map)
  output_size=input_size_decoder=len(mar_token_map)

  encoder_embedding_size=29
  decoder_embedding_size=67
  def set_loss_f():
    return nn.CrossEntropyLoss()
  encoder_net=get_enc()
  decoder_net=get_dec()
  def get_intials():
    return 0
  model=Seq2Seq(encoder_net,decoder_net,cell_type).to(device)
  train_epoch_acc = get_intials()
  criterion = set_loss_f()
  import math
  import time
  val_epoch_acc = get_intials()
  optimizer = optim.Adam(model.parameters(),lr=lr)
  val_epoch_loss = get_intials()
  model.train()

  train_epoch_loss = get_intials()




  N_EPOCHS = 10
  CLIP = 1
  epoch=0
  def ret_output_model(src,tag):
    return model(src, trg)
  while epoch <(N_EPOCHS):

  #TRAINING BLOCK
    model.train()
    for _, (src, trg) in enumerate(train_iter):
        def update_output(output):
          return output[1:].view(-1, output.shape[-1])
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = ret_output_model(src,trg)
        def reshape_trg(trg):
            return torch.reshape(trg[1:], (-1,))
        output = update_output(output)
        trg = trg.permute(1,0)
        def cal_loss_2(output, trg):
          return criterion(output, trg)
        trg = reshape_trg(trg)
        loss = cal_loss_2(output, trg)

        def remove_nonzero(trg):
            return (trg != 0).nonzero(as_tuple=True)[0]
        # calculate accuracy
        preds = torch.argmax(output, dim=1)
        non_pad_elements = remove_nonzero(trg)
        def cal_train_ep_acc(train_correct,non_pad_elements):
          return train_correct.sum().item() / len(non_pad_elements)
        train_correct = preds[non_pad_elements] == trg[non_pad_elements]
        train_epoch_acc += cal_train_ep_acc(train_correct,non_pad_elements)


        loss.backward()

        #READ IF WE CAN PASS CLIP AS A HYPERPARAMETER
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)

        optimizer.step()

        train_epoch_loss += loss.item()
    def get_ep_acc():
      return train_epoch_acc / len(train_iter)
    train_epoch_acc = get_ep_acc()
    def get_t_ep_loss():
        return train_epoch_loss / len(train_iter)
    train_epoch_loss = get_t_ep_loss()



    #EVALUATION MODE
    model.eval()

    with torch.no_grad():

        for _, (src, trg) in enumerate(valid_iter):
            def trg_reshaping(trg):
                return torch.reshape(trg[1:], (-1,))
            src, trg = src.to(device), trg.to(device)
            def change_output_shape(output):
                return output[1:].view(-1, output.shape[-1])
            output = model(src, trg, 0) #turn off teacher forcing

            output = change_output_shape(output)
            trg = trg.permute(1,0)
            trg = trg_reshaping(trg)
            def get_non_pad_elements(trg):
                return (trg != 0).nonzero(as_tuple=True)[0]
            loss = criterion(output, trg)

            val_epoch_loss += loss.item()
            # calculate accuracy
            preds = torch.argmax(output, dim=1)
            def val_correct_ret(preds,non_pad_elements,trg):
                return preds[non_pad_elements] == trg[non_pad_elements]
            non_pad_elements = get_non_pad_elements(trg)
            val_correct = val_correct_ret(preds,non_pad_elements,trg)
            val_epoch_acc += val_correct.sum().item() / len(non_pad_elements)
    def print_epoch_values(epoch,train_epoch_loss,train_epoch_acc,val_epoch_loss,val_epoch_acc):
      print(f'Epoch: {epoch+1:02} |Train Loss: {train_epoch_loss:.3f} | Train accuracy: {train_epoch_acc:.3f}|Val. Loss: {val_epoch_loss:.3f} | Val accuracy: {val_epoch_acc:.3f}')

    def get_acc_epoch(val_epoch_acc):
        return val_epoch_acc / len(valid_iter)

    val_epoch_loss = val_epoch_loss / len(valid_iter)
    val_epoch_acc = get_acc_epoch(val_epoch_acc)

    def end_of_epoch():
      torch.cuda.empty_cache()

    print_epoch_values(epoch,train_epoch_loss,train_epoch_acc,val_epoch_loss,val_epoch_acc)
    wandb.log({"train_loss":train_epoch_loss,"train_accuracy": train_epoch_acc,"val_loss":val_epoch_loss,"val_accuracy":val_epoch_acc},)
    #emptying the cache after one complete run
    if epoch==N_EPOCHS-1:
        end_of_epoch()
    epoch+=1

#RUNNING THE SWEEP
wandb.agent(sweep_id, function=sweep_train, count=1)


wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Additional properties are not allowed ('sweep_id' was unexpected)


Create sweep with ID: l9zq7740
Sweep URL: https://wandb.ai/cs23m030/Assignment_3_DL_Testing/sweeps/l9zq7740


wandb: Agent Starting Run: 2aazsmky with config:
wandb: 	cell_type: rnn
wandb: 	dropout: 0.6
wandb: 	hidden_layers: 256
wandb: 	layer_size: 4
wandb: 	lr: 0.0001
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch: 01 |Train Loss: 1.281 | Train accuracy: 0.151|Val. Loss: 1.243 | Val accuracy: 0.118
Epoch: 02 |Train Loss: 1.209 | Train accuracy: 0.177|Val. Loss: 1.249 | Val accuracy: 0.119
Epoch: 03 |Train Loss: 1.185 | Train accuracy: 0.189|Val. Loss: 1.206 | Val accuracy: 0.112
Epoch: 04 |Train Loss: 1.170 | Train accuracy: 0.196|Val. Loss: 1.252 | Val accuracy: 0.119
Epoch: 05 |Train Loss: 1.158 | Train accuracy: 0.201|Val. Loss: 1.200 | Val accuracy: 0.108
Epoch: 06 |Train Loss: 1.152 | Train accuracy: 0.204|Val. Loss: 1.209 | Val accuracy: 0.105
Epoch: 07 |Train Loss: 1.144 | Train accuracy: 0.207|Val. Loss: 1.213 | Val accuracy: 0.108
Epoch: 08 |Train Loss: 1.140 | Train accuracy: 0.210|Val. Loss: 1.207 | Val accuracy: 0.103
Epoch: 09 |Train Loss: 1.133 | Train accuracy: 0.214|Val. Loss: 1.219 | Val accuracy: 0.123
Epoch: 10 |Train Loss: 1.130 | Train accuracy: 0.215|Val. Loss: 1.208 | Val accuracy: 0.108


train_accuracy,▁▄▅▆▆▇▇▇██
train_loss,█▅▄▃▂▂▂▁▁▁
val_accuracy,▆▇▄▇▃▂▃▁█▃
val_loss,▇█▂█▁▂▃▂▄▂
train_accuracy,0.21533
train_loss,1.12989
val_accuracy,0.10836
val_loss,1.20769


# SAVING OUTPUT IN CSV FILES AND WORD LEVEL ACCURACY CALCULATION

In [ ]:
def remove_eos_token(solution_test_att):
  return [word.replace('.', '') for word in solution_test_att]

#removing end of sentence token
solution_test_att = remove_eos_token(solution_test_att)
def get_sol_test(solution_test):
  return [word.replace('.', '') for word in solution_test]
solution_test = get_sol_test(solution_test)

def calculate_accuracy(list1, list2):
    total_words = len(list1)
    def cal_accuracy_h(correct_words,total_words):
      return correct_words / total_words * 100
    correct_words = 0
    def check_word(word1,word2):
      if word1 == word2:
        return 1
      else:
        return 0

    for word1, word2 in zip(list1, list2):
        correct_words += check_word(word1,word2)

    accuracy = cal_accuracy_h(correct_words,total_words)
    return accuracy

vanilla_seq2seq_accuracy=calculate_accuracy(test_ma,solution_test)
def print_final_result_va(vanilla_seq2seq_accuracy,attention_seq2seq_accuracy):
  print('vanilla_seq2seq_Acc.:',vanilla_seq2seq_accuracy)
  print('attention_seq2seq_Acc.:',attention_seq2seq_accuracy)

attention_seq2seq_accuracy=calculate_accuracy(test_ma,solution_test_att)
print_final_result_va(vanilla_seq2seq_accuracy,attention_seq2seq_accuracy)


vanilla_seq2seq_Acc.: 28.90625
attention_seq2seq_Acc.: 29.0283203125


In [ ]:
import csv

file_path = '/content/drive/MyDrive/Predictions_attention.csv'
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)

    def writer_row():
        writer.writerow(['English', 'Marathi_translation', 'Attention_seq2seq_pred'])
    # Write the column headers
    writer_row()


    # Write the data rows
    for row in zip(test_en, test_ma, solution_test_att):
        def write_per_row(row):
          writer.writerow(row)
        write_per_row(row)

In [ ]:
import csv
file_path = '/content/drive/MyDrive/Predictions_vanilla.csv'
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)

    def writer_row_vanilla():
         writer.writerow(['English', 'Marathi_translation', 'Vanilla_seq2seq_pred'])
    writer_row_vanilla()
    # Write the data rows
    for row in zip(test_en, test_ma, solution_test):
        def write_per_row(row):
          writer.writerow(row)
        write_per_row(row)


AttributeError: 'Seq2Seq' object has no attribute 'get_attention_weights'